In [17]:
import pandas as pd
import pickle

In [18]:
from FedTools import MonetaryPolicyCommittee
from FedTools import BeigeBooks
from FedTools import FederalReserveMins

In [19]:
dataset = MonetaryPolicyCommittee().find_statements()

Constructing links between 1994 and 2022
Extracting the past 199 FOMC Statements.
Retrieving articles.
.......................................................................................................................................................................................................

In [60]:
dataset1 = dataset

In [20]:
dataset1.columns

Index(['FOMC_Statements'], dtype='object')

In [21]:
import re
title_strip = re.compile(r"(.*?)EDT[\s]+Share")
end_voting_trip = re.compile(r"Voting for the monetary policy action were.*")

In [71]:
dataset1["FOMC_Statements"] = dataset1["FOMC_Statements"].apply(lambda x : re.sub(r"(.*?)EDT[\s]+Share", "", str(x)))
dataset1["FOMC_Statements"] = dataset1["FOMC_Statements"].apply(lambda x : re.sub(r"Voting for the monetary policy action were.*", "", str(x)))
dataset1["FOMC_Statements"] = dataset1["FOMC_Statements"].apply(lambda x : re.sub(r"Home | News.*", "", str(x)))

In [22]:
dataset1.to_csv("FOMC_Statements_cleaned.csv")

with open('FOMC_Statements_cleaned.pickle', 'wb') as handle:
    pickle.dump(dataset1, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [78]:
with open('FOMC_Statements_cleaned.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [23]:
dataset1

,FOMC_Statements
1994-02-04,Chairman Alan Greenspan announced today that t...
1994-03-22,Chairman Alan Greenspan announced today that t...
1994-04-18,Chairman Alan Greenspan announced today that t...
1994-05-17,The Federal Reserve today announced two action...
1994-08-16,The Federal Reserve announced today the follow...
...,...
2022-01-26,"The Federal Reserve, the central bank of the U..."
2022-03-16,Indicators of economic activity and employme...
2022-05-04,Although overall economic activity edged dow...
2022-06-15,Overall economic activity appears to have pi...


## Ready LDA

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20220034\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import gensim, spacy
import gensim.corpora as corpora
from nltk.corpus import stopwords

import pandas as pd
import re
from tqdm import tqdm
import time


import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

## Setup nlp for spacy
nlp = spacy.load("en_core_web_sm")

# Load NLTK stopwords
stop_words = stopwords.words('english')
# Add some extra words in it if required
stop_words.extend(['from', 'subject', 'use','pron'])

C:\Users\20220034\dev\CBankVenv\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [108]:
# Convert into list
data = dataset1.FOMC_Statements.values.tolist()

### Cleaning data


# Remove new line characters and extra space
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove single quotes
data = [re.sub("'", "", sent) for sent in data]

### Lemmatization
data_lemma = []
for txt in tqdm(data):
    lis = []
    doc = nlp(txt)
    for token in doc:
        lis.append(token.lemma_)
    data_lemma.append(' '.join(lis))

### Tokenization and gensim stopword removal

# You can look for all gensim stopwords by running -> 'gensim.parsing.preprocessing.STOPWORDS'

# Function to tokenize
# Also remove words whose length less than 3 (you can chang it)
def tokenization_with_gen_stop(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)

    return result

## Apply tokenization function
data_words = []
for txt in tqdm(data_lemma):
    data_words.append(tokenization_with_gen_stop(txt))

### NLTK Stopword removal (extra stopwords)

data_words_clean = []
for word in tqdm(data_words):
    wrd = []
    for w in word:
        if w not in stop_words:
            wrd.append(w)
    data_words_clean.append(wrd)

100%|██████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 1184.15it/s]


In [110]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words_clean)
# Print dictionary
print(dictionary.token2id)

## Create Term document frequency (corpus)
# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in data_words_clean]
# Print corpus for first document
print(corpus[0])

{'accommodative': 0, 'action': 1, 'alan': 2, 'announce': 3, 'associate': 4, 'avoid': 5, 'chairman': 6, 'committee': 7, 'committees': 8, 'condition': 9, 'decide': 10, 'decision': 11, 'degree': 12, 'early': 13, 'economic': 14, 'enhance': 15, 'expansion': 16, 'expect': 17, 'fact': 18, 'federal': 19, 'firming': 20, 'greenspan': 21, 'immediately': 22, 'increase': 23, 'market': 24, 'monetary': 25, 'money': 26, 'open': 27, 'order': 28, 'policy': 29, 'position': 30, 'pressure': 31, 'purpose': 32, 'rate': 33, 'reserve': 34, 'short': 35, 'slightly': 36, 'small': 37, 'stance': 38, 'sustain': 39, 'term': 40, 'today': 41, 'accommodation': 42, 'adjustment': 43, 'agree': 44, 'allow': 45, 'appropriate': 46, 'approve': 47, 'atlanta': 48, 'bank': 49, 'banks': 50, 'board': 51, 'boards': 52, 'borrow': 53, 'boston': 54, 'charge': 55, 'chicago': 56, 'city': 57, 'combine': 58, 'completely': 59, 'continue': 60, 'dallas': 61, 'depository': 62, 'design': 63, 'development': 64, 'directors': 65, 'discount': 66, '

In [111]:
# Easy to observe format of corpus
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accommodative', 1),
  ('action', 2),
  ('alan', 1),
  ('announce', 2),
  ('associate', 1),
  ('avoid', 1),
  ('chairman', 2),
  ('committee', 2),
  ('committees', 1),
  ('condition', 1),
  ('decide', 2),
  ('decision', 1),
  ('degree', 1),
  ('early', 1),
  ('economic', 1),
  ('enhance', 1),
  ('expansion', 1),
  ('expect', 1),
  ('fact', 1),
  ('federal', 1),
  ('firming', 1),
  ('greenspan', 2),
  ('immediately', 1),
  ('increase', 2),
  ('market', 3),
  ('monetary', 1),
  ('money', 1),
  ('open', 1),
  ('order', 1),
  ('policy', 1),
  ('position', 1),
  ('pressure', 1),
  ('purpose', 1),
  ('rate', 1),
  ('reserve', 2),
  ('short', 1),
  ('slightly', 1),
  ('small', 1),
  ('stance', 1),
  ('sustain', 1),
  ('term', 1),
  ('today', 1)]]

## Train

In [117]:
start_time = time.time()
##
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary,random_state=100,passes=10)
# Saving trained model
ldamodel.save('LDA_NYT')
# Loading trained model
ldamodel = gensim.models.ldamodel.LdaModel.load('LDA_NYT')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.505099058151245 seconds ---


In [121]:
start_time = time.time()
##
## Multicore LDA
NUM_TOPICS = 5
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = NUM_TOPICS, id2word=dictionary,random_state=100,passes=10)
# Saving trained model
lda_multicore_model.save('LDA_NYT_multicore')
# Loading trained model
lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))

--- 10.024554014205933 seconds ---


In [122]:
ldamodel.print_topics(-1)

[(0,
  '0.031*"federal" + 0.027*"rate" + 0.019*"reserve" + 0.016*"growth" + 0.016*"economic" + 0.016*"committee" + 0.015*"discount" + 0.013*"action" + 0.013*"document" + 0.012*"board"'),
 (1,
  '0.030*"policy" + 0.027*"monetary" + 0.025*"committee" + 0.015*"chairman" + 0.014*"economic" + 0.014*"growth" + 0.013*"federal" + 0.013*"action" + 0.011*"believe" + 0.011*"risk"'),
 (2,
  '0.028*"federal" + 0.024*"bank" + 0.023*"reserve" + 0.019*"market" + 0.018*"financial" + 0.014*"data" + 0.014*"policy" + 0.012*"monetary" + 0.012*"committee" + 0.011*"action"'),
 (3,
  '0.027*"committee" + 0.025*"inflation" + 0.018*"market" + 0.017*"federal" + 0.016*"policy" + 0.016*"economic" + 0.016*"financial" + 0.014*"rate" + 0.012*"data" + 0.011*"monetary"'),
 (4,
  '0.023*"policy" + 0.023*"committee" + 0.021*"action" + 0.017*"monetary" + 0.014*"inflation" + 0.014*"document" + 0.014*"price" + 0.014*"federal" + 0.011*"approve" + 0.011*"board"')]

In [123]:
# Compute Perplexity Score
print('nPerplexity Score: ', ldamodel.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=ldamodel, texts=data_words_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('nCoherence Score: ', coherence_lda)

nPerplexity Score:  -5.760640218928027
nCoherence Score:  0.49861073613410467


In [124]:
# To plot at Jupyter notebook
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
# Save pyLDA plot as html file
pyLDAvis.save_html(lda_viz, 'LDA_NYT.html')
lda_viz

C:\Users\20220034\dev\CBankVenv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.191352 -0.034350       1        1  37.088077
0     -0.084301  0.148029       2        1  21.304889
4     -0.137788 -0.008975       3        1  16.265186
1     -0.135758 -0.116618       4        1  14.663061
2      0.166495  0.011914       5        1  10.678787, topic_info=             Term         Freq        Total Category  logprob  loglift
49           bank   484.000000   484.000000  Default  30.0000  30.0000
1296         data   533.000000   533.000000  Default  29.0000  29.0000
34        reserve   987.000000   987.000000  Default  28.0000  28.0000
70      financial   804.000000   804.000000  Default  27.0000  27.0000
29         policy  1582.000000  1582.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
249     governors    49.007776   282.807253   Topic5  -5.2933   0.4841
73      inflation    56.069163  1363.686410   Topic5  -5.1587  -0.9544
64    development    46.275513   275.312531   Topic5  -5.3507   0.4536
259         price    47.832225   768.836666   Topic5  -5.3176  -0.5403
60       continue    46.313629   702.382094   Topic5  -5.3498  -0.4821

[414 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
151       1  0.003330  accessibility
151       2  0.236437  accessibility
151       3  0.286388  accessibility
151       4  0.469543  accessibility
151       5  0.003330  accessibility
...     ...       ...            ...
600       5  0.006919         weight
1563      1  0.953854           wide
1563      5  0.037406           wide
542       2  0.965881         worker
542       3  0.028408         worker

[869 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])

In [104]:
re.sub(r'\s+', '', "sequence   ")

'sequence'

## Mallet

In [5]:
!pip install little_mallet_wrapper 
#https://github.com/maria-antoniak/little-mallet-wrapper/blob/master/demo.ipynb

In [6]:
!pip install seaborn

In [5]:
from collections import defaultdict
from datetime import datetime
import math
from operator import itemgetter
import os
import random
import re

import numpy as np
import pandas as pd

%matplotlib inline

import little_mallet_wrapper as lmw

In [6]:
os.environ.update({'MALLET_HOME':r'C:\\mallet-2.0.8'})

In [7]:
with open('FOMC_Statements_cleaned.pickle', 'rb') as handle:
    dataset1 = pickle.load(handle)

In [8]:
training_data = [lmw.process_string(t) for t in dataset1['FOMC_Statements'].tolist()]

In [9]:

path_to_mallet = 'C:\\mallet-2.0.8\\bin\\mallet'

In [10]:
len(training_data)

199

In [11]:
lmw.print_dataset_stats(training_data)

Number of Documents: 199
Mean Number of Words per Document: 478.5
Vocabulary Size: 2544


In [12]:
num_topics = 5  # CHANGE THIS TO YOUR PREFERRED NUMBER OF TOPICS
output_directory_path = '\\Users\\20220034\\Desktop\\lmw_output'

path_to_training_data           = output_directory_path + '\\training.txt'
path_to_formatted_training_data = output_directory_path + '\\mallet.training'
path_to_model                   = output_directory_path + '\\mallet.model.' + str(num_topics)
path_to_topic_keys              = output_directory_path + '\\mallet.topic_keys.' + str(num_topics)
path_to_topic_distributions     = output_directory_path + '\\mallet.topic_distributions.' + str(num_topics)
path_to_word_weights            = output_directory_path + '\\mallet.word_weights.' + str(num_topics)
path_to_diagnostics             = output_directory_path + '\\mallet.diagnostics.' + str(num_topics) + '.xml'

In [13]:

lmw.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


In [14]:
lmw.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)


Training topic model...
Complete


In [15]:
topic_keys = lmw.load_topic_keys(output_directory_path + '\\mallet.topic_keys.' + str(num_topics))

In [16]:
topic_keys = lmw.load_topic_keys(output_directory_path + '/mallet.topic_keys.' + str(num_topics))

for i, t in enumerate(topic_keys):
    print(i, '\t', ' '.join(t[:10]))

0 	 document var style committee handler cpo function inflation onreadystatechange appendchild
1 	 financial data federal committee bank reserve policy monetary economic market
2 	 federal rate reserve economic discount board approved action growth percent
3 	 committee inflation market economic percent rate conditions federal labor policy
4 	 policy monetary committee action chairman federal economic growth inflation voting
